In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
df = pd.read_pickle(f'{name_df}.pkl')

In [3]:
df

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_191,Col_192,Col_193,Col_194,Col_195,Col_196,Col_197,Col_198,Col_199,Col_200
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41361,1,1,1,1,1,1,1,1,1,1,...,0,1,1,0,0,0,0,0,1,0
41362,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,1,1,0
41363,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,1,0,0,0
41364,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,1,0,0,0,0


In [4]:
len(df)

41366

In [5]:
new_columns = [f'Col_{i}' for i in range(1, 50 + 1)]
df_sort = df.sort_values(by = new_columns).reset_index(drop=True)

In [6]:
# Check the current data types of the DataFrame
print("Before Conversion:")
print(df_sort.dtypes)
print("\n")

for i in df_sort.columns:
    df_sort[i] = df_sort[i].astype(int)

# Check the data types after conversion
print("After Conversion:")
print(df_sort.dtypes)

Before Conversion:
Col_1      int32
Col_2      int32
Col_3      int32
Col_4      int32
Col_5      int32
           ...  
Col_196    int32
Col_197    int32
Col_198    int32
Col_199    int32
Col_200    int32
Length: 200, dtype: object


After Conversion:
Col_1      int32
Col_2      int32
Col_3      int32
Col_4      int32
Col_5      int32
           ...  
Col_196    int32
Col_197    int32
Col_198    int32
Col_199    int32
Col_200    int32
Length: 200, dtype: object


In [7]:
len(df_sort.columns)

200

In [8]:
# Step 1: Prepare Data
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  37229
len x test:  4137
len y train:  37229
len y test:  4137
10159 72


,train,test
accuracy,0.999678,0.755378
precision,1.000000,0.425000
recall,0.998423,0.470046
f1,0.999211,0.446389


In [9]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\forward_model\\{SIZE}\\decision_tree_size{SIZE}_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.821913  0.820885
precision  0.611695  0.630390
recall     0.351998  0.353687
f1         0.446855  0.453137
63 5
              train      test
accuracy   0.825996  0.829345
precision  0.640547  0.680000
recall     0.338460  0.352535
f1         0.442896  0.464340
127 6
              train      test
accuracy   0.829058  0.830070
precision  0.675607  0.710997
recall     0.314537  0.320276
f1         0.429238  0.441620
255 7
              train      test
accuracy   0.834645  0.829587
precision  0.675278  0.673774
recall     0.367639  0.364055
f1         0.476085  0.472700
493 8
              train      test
accuracy   0.841038  0.830312
precision  0.704600  0.675847
recall     0.382492  0.367512
f1         0.495826  0.476119
921 9
              train      test
accuracy   0.851648  0.826203
precision  0.714727  0.629565
recall     0.456099  0.417051
f1         0.556848  0.501733
1583 10
              train      test
accuracy   0.866180  0.81870

In [10]:
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 20, 42)
    name_file = f"{PATH_MODELS}\\forward\\{SIZE}\\dt_size{SIZE}_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
9091 20
              train      test
accuracy   0.989900  0.759246
precision  0.986592  0.414062
recall     0.963846  0.440618
f1         0.975086  0.426928
1
9075 20
              train      test
accuracy   0.989927  0.778825
precision  0.993416  0.474330
recall     0.958699  0.489068
f1         0.975749  0.481586
2
8909 20
              train      test
accuracy   0.988154  0.767706
precision  0.982229  0.459954
recall     0.961007  0.451178
f1         0.971502  0.455524
3
9215 20
              train      test
accuracy   0.991834  0.769640
precision  0.988353  0.463705
recall     0.973157  0.483070
f1         0.980696  0.473190
4
8923 20
              train      test
accuracy   0.989497  0.776408
precision  0.982390  0.445783
recall     0.965260  0.492736
f1         0.973750  0.468085
5
8849 20
              train      test
accuracy   0.991109  0.766497
precision  0.985308  0.446903
recall     0.971294  0.464368
f1         0.978251  0.455468
6
9153 20
              train      test
